In [1]:
import numpy as np

In [4]:
def policy_evaluation(pi, P, gamma=1.0, thetha=1e-10):
    
    prev_V = np.zeros(len(P))
    
    while True:
        V = np.zeros(len(P))
        
        for s in range(len(P)):
            
            for prob, next_state, reward, done in P[s][pi(s)]:
                ## Each transition tuple has a probability, next state, reward, and a done flag
                ## Indicating whether the 'next_state' is terminal or not
                V[s] += prob * (reward + gamma*prev_V[next_state] * (not done))
                ## Done flag  is used to ensure the value of the next_state when landing on the terminal
                ##state is zero. We don't want infinite sum
        if np.max(np.abs(prev_V - V)) < theta:
            break
        
        prev_V = V.copy()
    
    return V
                  